## Stretch effect

In [ ]:
from PIL import Image
import math

def gradual_stretch(
    input_path: str,
    output_path: str,
    x_start: int,
    width_multiplier: float = 2.0,
    affected_columns: int = 10,
    growth_factor: float = 1.5
):
    img = Image.open(input_path).convert("RGBA")
    width, height = img.size
    new_width = int(width * width_multiplier)

    if not (0 <= x_start < width):
        raise ValueError("x_start must be within image bounds")

    out = Image.new("RGBA", (new_width, height))

    # Copy unchanged left portion
    out.paste(img.crop((0, 0, x_start, height)), (0, 0))

    write_x = x_start
    remaining_width = new_width - x_start

    last_col = None

    # Generate stretch widths using exponential growth
    stretch_widths = [
        max(1, int(math.pow(growth_factor, i)))
        for i in range(affected_columns)
    ]

    for i, target_width in enumerate(stretch_widths):
        if remaining_width <= 0:
            break

        src_x = min(x_start + i, width - 1)
        col = img.crop((src_x, 0, src_x + 1, height))
        last_col = col

        w = min(target_width, remaining_width)
        stretched = col.resize((w, height), Image.NEAREST)
        out.paste(stretched, (write_x, 0))

        write_x += w
        remaining_width -= w

    # Fill all remaining space with final column
    if remaining_width > 0 and last_col is not None:
        stretched = last_col.resize((remaining_width, height), Image.NEAREST)
        out.paste(stretched, (write_x, 0))

    out.save(output_path)
    print(f"Saved stretched image to {output_path}")


gradual_stretch(
    input_path="img/input1.jpg",
    output_path="img/output1.2.png",
    x_start=960,
    width_multiplier=1,
    affected_columns=40,
    growth_factor=1.1
)

Saved stretched image to output1.2.png


## Horizontal blur effect to fix clear sections

In [ ]:
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm

def horizontal_gradient_blur(
    input_path,
    output_path,
    start_x,
    growth_multiplier
):
    img = Image.open(input_path).convert("RGBA")
    arr = np.array(img, dtype=np.float32)

    height, width, channels = arr.shape
    output = arr.copy()

    for y in tqdm(range(height), desc="Applying horizontal blur"):
        for x in range(start_x, width):
            radius = int((x - start_x) * growth_multiplier)

            if radius <= 0:
                continue

            left = max(0, x - radius)
            right = min(width - 1, x + radius)

            output[y, x] = arr[y, left:right + 1].mean(axis=0)

    result = Image.fromarray(np.clip(output, 0, 255).astype(np.uint8))
    result.save(output_path)

horizontal_gradient_blur(
    "img/output1.2.png",
    "img/output1.2_blurred.png",
    960 - 10,
    0.05
)

Applying horizontal blur:   0%|          | 0/2949 [00:00<?, ?it/s]

In [ ]:
gradual_stretch(
    input_path="img/maxim-tolchinskiy-GA2y1XU9bIw-unsplash.jpg",
    output_path="img/output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    x_start=1165,
    width_multiplier=1,
    affected_columns=100,
    growth_factor=1.1
)

Saved stretched image to output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png


In [ ]:
horizontal_gradient_blur(
    "img/output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    "img/output-maxim-tolchinskiy-GA2y1XU9bIw-unsplash.png",
    1165 - 10,
    0.05
)

Applying horizontal blur:   0%|          | 0/2880 [00:00<?, ?it/s]

## Gradient algorithm instead?

In [45]:
from PIL import Image
import numpy as np
import math
from tqdm.notebook import tqdm

def gradient_stretch(
    input_path: str,
    output_path: str,
    x_start: int,
    width_multiplier: float = 2.0,
    affected_columns: int = 40,
    growth_factor: float = 1.1,
    show_progress: bool = True
):
    img = Image.open(input_path).convert("RGBA")
    src = np.array(img, dtype=np.float32)

    h, w, c = src.shape
    new_w = int(w * width_multiplier)

    if not (0 <= x_start < w):
        raise ValueError("x_start must be within image bounds")

    out = np.zeros((h, new_w, c), dtype=np.float32)

    # Copy unchanged left portion
    out[:, :x_start] = src[:, :x_start]

    # --- 1. Generate source columns with increasing spacing ---
    src_xs = []
    cur_x = x_start
    spacing = 1.0

    for _ in range(affected_columns):
        ix = int(round(cur_x))
        if ix >= w:
            break
        src_xs.append(ix)
        spacing *= growth_factor
        cur_x += spacing

    if len(src_xs) < 2:
        raise ValueError("Not enough source columns generated")

    # --- 2. Map to output positions ---
    # Compute source spacings
    src_deltas = [src_xs[i + 1] - src_xs[i] for i in range(len(src_xs) - 1)]
    src_deltas.append(src_deltas[-1])  # extend last spacing

    # Normalize spacings to output width
    total_src_span = sum(src_deltas)
    scale = (new_w - x_start) / total_src_span

    out_xs = [x_start]
    acc = x_start

    for d in src_deltas[:-1]:
        acc += d * scale
        out_xs.append(int(round(acc)))


    # --- 3. Interpolate gradients ---
    pairs = range(len(src_xs) - 1)
    iterator = tqdm(pairs, desc="Interpolating columns") if show_progress else pairs

    for i in iterator:
        sx0, sx1 = src_xs[i], src_xs[i + 1]
        ox0, ox1 = out_xs[i], out_xs[i + 1]

        col0 = src[:, sx0]
        col1 = src[:, sx1]

        span = max(1, ox1 - ox0)

        for dx in range(span):
            t = dx / span
            out[:, ox0 + dx] = (1 - t) * col0 + t * col1

    # --- 4. Fill remainder with final column ---
    last_src_x = src_xs[-1]
    last_out_x = out_xs[-1]
    last_col = src[:, last_src_x]

    if last_out_x < new_w:
        out[:, last_out_x:] = last_col[:, None]

    result = Image.fromarray(np.clip(out, 0, 255).astype(np.uint8))
    result.save(output_path)

    print(f"Saved gradient-stretched image to {output_path}")

In [ ]:
gradient_stretch(
    input_path="img/input1.jpg",
    output_path="img/output_gradient (old but cool look).png",
    x_start=960,
    width_multiplier=1,
    affected_columns=20,
    growth_factor=1.5
)

Interpolating columns:   0%|          | 0/14 [00:00<?, ?it/s]

Saved gradient-stretched image to output_gradient (old but cool look).png
